In [1]:
# reloading modules if needed
from importlib import reload

# first import the whole package
import PyAPD

# then load all the functions in the .py files in the .../PyAPD/ directory:
from PyAPD import *

torch.set_printoptions(precision=10) # show more digits

#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
dt = torch.float32
torch.set_default_dtype(dt)
#torch.set_default_device(device)

In [2]:
import copy

roe = 0.0

# D = 2  # Dimension of the ambient space
# #Ms = [1200]
# Ns = [25,50,100,250,500,1000,2500,5000]

D = 3  # Dimension of the ambient space
# Ms = [200]
Ns = [50,100,250,500,1000,2500,5000,10000]



apd = apd_system(N=Ns[0], D=D,dt=torch.float32,device = device,
                  pixel_size_prefactor = 2,
                  ani_thres = 0.3,
                  radius_of_exclusion = roe,
                  heuristic_W = False)


dts = [torch.float32] # single precision or double precision
anisotropies = [0.0, 0.3, 0.7]
seeds = [i for i in range(10)]

total_no_data_points = len(Ns)*len(anisotropies)*len(seeds)
print(*Ns)
print(*dts)
print(*anisotropies)
print(*seeds)

50 100 250 500 1000 2500 5000 10000
torch.float32
0.0 0.3 0.7
0 1 2 3 4 5 6 7 8 9


In [3]:
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("FAKE FIRST RUN")

kk=-1
for seed in seeds[:1]:
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("Random seed =", seed)
    apd.seed = seed
    for dt in dts[:1]:
        torch.set_default_dtype(dt)
        apd.dt = dt        
        for ani_thres in anisotropies[:1]:
            apd.ani_thres = ani_thres
            for N in Ns[:1]:
                kk+= 1
                apd.optimality = False
                apd.N = N
                
                ### SAMPLING
                # to ensure consistenscy, we want to do the sampling in single precision
                torch.set_default_dtype(torch.float32)
                
                ### sampling seed points
                torch.manual_seed(seed)
                X = sample_seeds_with_exclusion(N,dim=D,
                                                radius_prefactor = apd.radius_of_exclusion,
                                                verbose=True)
                
                ### sampling anisotropy matrices
                torch.manual_seed(seed+10)
                As = sample_normalised_spd_matrices(N,dim = D, ani_thres = ani_thres)
                
                ### back to the precision we actually want to work with
                ### (when populating the class, the conversion will take place automatically):
                torch.set_default_dtype(apd.dt)
                apd.set_X(X=X, verbose=True)
                apd.set_As(As=As)
                
                apd.set_target_masses()
                apd.set_W()
                apd.set_pixel_params()
                apd.assemble_apd(verbose=True,record_time = True, backend = "CPU")
                print(kk)
                print(apd.seed)
                
                #d = apd.__dict__
                #apd.data[kk] = copy.copy({i:d[i] for i in d if i!='data'})

                
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("ACTUAL RUN")
kk=-1
for seed in seeds:
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("Random seed =", seed)
    apd.seed = seed
    for dt in dts:
        torch.set_default_dtype(dt)
        apd.dt = dt        
        for ani_thres in anisotropies:
            apd.ani_thres = ani_thres
            for N in Ns:
                kk+= 1
                print("Populating data point",kk+1, " / ", total_no_data_points)
                apd.optimality = False
                apd.N = N
                ### SAMPLING
                # to ensure consistenscy, we want to do the sampling in single precision
                torch.set_default_dtype(torch.float32)
                
                ### sampling seed points
                torch.manual_seed(seed)
                X = sample_seeds_with_exclusion(N,dim=D,
                                                radius_prefactor = apd.radius_of_exclusion,
                                               verbose = True)
                
                ### sampling anisotropy matrices
                torch.manual_seed(seed+10)
                As = sample_normalised_spd_matrices(N,dim = D, ani_thres = ani_thres)
                
                ### back to the precision we actually want to work with
                ### (when populating the class, the conversion will take place automatically):
                torch.set_default_dtype(apd.dt)
                apd.set_X(X=X)
                apd.set_As(As=As)
                
                apd.set_target_masses()
                apd.set_W()
                apd.set_pixel_params()
                apd.assemble_pixels()
                apd.assemble_apd(verbose=True,record_time = True, backend = "CPU")
                apd.Y = None
                apd.PS = None
                apd.y = None
                apd.a = None
                apd.x = None
                apd.W = None
                apd.w = None
                
                d = apd.__dict__
                apd.data[kk] = copy.copy({i:d[i] for i in d if i!='data'})


import pickle

file = f'apd_{D}D_radius_{roe}_CPU_perlmutter.pkl'

with open(file, "wb") as fp:   #Pickling
     pickle.dump(apd, fp)

# import bz2
# import pickle
# import _pickle as cPickle

# def compressed_pickle(title, data):
#     with bz2.BZ2File(title + '.pbz2', 'w') as f: 
#         cPickle.dump(data, f)
# compressed_pickle("test",apd.data)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
FAKE FIRST RUN
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Random seed = 0
0 proposed seed points have been excluded.
[KeOps] Generating code for formula ArgMin_Reduction((Var(0,3,1)-Var(1,3,0))|MatVecMult(Var(2,9,0),Var(0,3,1)-Var(1,3,0))-Var(3,1,0),1) ... OK
[pyKeOps] Compiling pykeops cpp 1034d3d43d module ... OK
APD generated in: 6.001694440841675 seconds.
0
0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ACTUAL RUN
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Random seed = 0
Populating data point 1  /  240
0 proposed seed points have been excluded.
APD generated in: 0.01525425910949707 seconds.
Populating data point 2  /  240
0 proposed seed points have been excluded.
APD generated in: 0.05553793907165527 seconds.
Populating data point 3  /  240
0 proposed seed points have been excluded.
APD generated in: 0.35384511947631836 seconds.
Populating data point 4  /  240
0 proposed seed points have been e

In [4]:
apd.radius_of_exclusion

0.0

In [5]:
apd.data[0]["X"].device

device(type='cpu')

In [6]:
apd.assemble_pixels()

In [7]:
apd.y